Reference: https://www.kaggle.com/kernels/scriptcontent/7827075/download

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import json
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import torch
is_torchvision_installed = True
try:
    import torchvision
except:
    is_torchvision_installed = False
import torch.utils.data
import random

class BalancedBatchSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset, labels=None):
        self.labels = labels
        self.dataset = dict()
        self.balanced_max = 0
        # Save all the indices for all the classes
        for idx in range(0, len(dataset)):
            label = dataset[idx][1]
            if label not in self.dataset:
                self.dataset[label] = list()
            self.dataset[label].append(idx)
            self.balanced_max = len(self.dataset[label]) \
                if len(self.dataset[label]) > self.balanced_max else self.balanced_max
        
        # Oversample the classes with fewer elements than the max
        for label in self.dataset:
            while len(self.dataset[label]) < self.balanced_max:
                self.dataset[label].append(random.choice(self.dataset[label]))
        self.keys = list(self.dataset.keys())
        self.currentkey = 0
        self.indices = [-1]*len(self.keys)

    def __iter__(self):
        while self.indices[self.currentkey] < self.balanced_max - 1:
            self.indices[self.currentkey] += 1
            yield self.dataset[self.keys[self.currentkey]][self.indices[self.currentkey]]
            self.currentkey = (self.currentkey + 1) % len(self.keys)
        self.indices = [-1]*len(self.keys)
    

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import autograd
import numpy as np
from torch.utils.data import DataLoader


class CramerRaoLowerBound:

    def __init__(self, model, crit, penalty, weight_decay_param, lr=0.001, weight=1000000):
        self.model = model
        self.weight = weight
        self.crit = crit
        if penalty in ('SGD', 'CRLBO'):
            self.optimizer = optim.SGD(self.model.parameters(), lr)
        elif penalty == 'L2':
            self.optimizer = optim.SGD(self.model.parameters(), lr, weight_decay=weight_decay_param)

    def forward_backward_update(self, input, target, penalty):
        output = self.model(input.float().cuda()).detach().requires_grad_(True)
        if penalty in ('SGD', 'L2'):
            loss = self.crit(output, target)
        elif penalty == 'CRLBO':
            loss = self._compute_consolidation_loss(self.weight) + self.crit(output, target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def save(self, filename):
        torch.save(self.model, filename)

    def load(self, filename):
        self.model = torch.load(filename)
        
    def _update_mean_params(self):
        for param_name, param in self.model.named_parameters():
            _buff_param_name = param_name.replace('.', '__')
            self.model.register_buffer(_buff_param_name+'_estimated_mean', param.data.clone())

    def _update_fisher_params(self, current_ds, batch_size, num_batch):
        dl = DataLoader(current_ds, batch_size = 50, shuffle=True)
        log_liklihoods = []
        for i, (input, target) in enumerate(dl):
            if i >= num_batch:
                break
            output = F.log_softmax(self.model(input.cuda().float()).detach().requires_grad_(True), dim=1)
            log_liklihoods.append(output[:, target])
        log_likelihood = torch.cat(log_liklihoods).mean()
        grad_log_liklihood = autograd.grad(log_likelihood, self.model.parameters(), allow_unused=True)
        _buff_param_names = [param[0].replace('.', '__') for param in self.model.named_parameters()]
        if(grad_log_liklihood[0] is not None):
            for _buff_param_name, param in zip(_buff_param_names, grad_log_liklihood):
                self.model.register_buffer(_buff_param_name+'_estimated_fisher', param.data.clone() ** 2)

    def register_crlbo_params(self, dataset, batch_size, num_batches):
        self._update_fisher_params(dataset, batch_size, num_batches)
        self._update_mean_params()

    def _compute_consolidation_loss(self, weight):
        try:
            losses = []
            for param_name, param in self.model.named_parameters():
                _buff_param_name = param_name.replace('.', '__')
                estimated_mean = getattr(self.model, '{}_estimated_mean'.format(_buff_param_name))
                estimated_fisher = getattr(self.model, '{}_estimated_fisher'.format(_buff_param_name))
                losses.append((estimated_fisher * (param - estimated_mean) ** 2).sum())
            return (weight / 2) * sum(losses)
        except AttributeError:
            return 0


Please visit the official website to find the dataset: https://www.cs.toronto.edu/~kriz/cifar.html

In [0]:
import torchvision
import torch

from torchvision import datasets, transforms

print('Loading data...')

# Image preprocessing, because VGG uses 224 * 224 size pictures, but CIFAR10 only has 32 * 32. In order to run the results faster,
# We enlarged them to 96 * 96 instead of 224 * 224 of the original paper
transform = transforms.Compose([
    transforms.Resize(96),# Scale to 96 * 96 size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalized
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                              std=[0.229, 0.224, 0.225])
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True,
                                        transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

print('Loading is Done!')

Loading data...



Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Loading is Done!


In [0]:
print('Total Images: ', len(trainset))

Total Images:  50000


In [0]:
num_classes = len(trainset.classes)
print('No. of Classes: ', num_classes)

No. of Classes:  10


In [0]:
print('Classes: ', trainset.class_to_idx)

Classes:  {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


In [0]:
class_sample_counts = np.bincount(trainset.targets)
print(class_sample_counts)

[5000 5000 5000 5000 5000 5000 5000 5000 5000 5000]


### Viewing the datasets

In [0]:
# fig = plt.figure(figsize=(14,10))

# for n in range(1, 10):
#   fig.add_subplot(4, 7, n)
#   img = trainset.data[n]
#   plt.imshow(img)
#   plt.title(trainset.classes[trainset.targets[n]])
#   plt.axis('off')

In [0]:
len(trainset)

50000

In [0]:
batches = 20

train_batch_size_ = int(len(trainset) / batches)
print('Train Batch size: ', train_batch_size_)

Train Batch size:  2500


In [0]:
test_batch_size_ = int(len(testset) / batches)
print('Test Batch size', test_batch_size_)

Test Batch size 500


In [0]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=train_batch_size_,sampler=BalancedBatchSampler(trainset, list(trainset.class_to_idx.values())), shuffle=False, num_workers = 4)
test_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size_, sampler=BalancedBatchSampler(testset, list(testset.class_to_idx.values())), shuffle=False, num_workers = 4)



In [0]:
for images, labels in train_loader:
  print(images.shape)
  break

torch.Size([2500, 3, 96, 96])


## Load and Compile the Model

In [0]:
from torch.autograd import Variable

# Hyperparameter
EPOCH = 100
learning_rate = 0.001

# Whether to use GPU
use_gpu = True


class VGG(nn.Module):
    '''
    VGG model
    '''
    def __init__(self, conv_features):
        super(VGG, self).__init__()
        self.conv_features = conv_features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(73728, 512), 
            nn.ReLU(True),
            nn.Dropout(),
            # nn.Linear(512, 512),
            # nn.ReLU(True),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.conv_features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Build the conv layer of the loop
def make_layers(struct, in_channels=1, batch_norm=False):
    layers = []
    for out_channels in struct:
        if out_channels == 'pooling':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = out_channels
    return nn.Sequential(*layers)


In [0]:
crit = nn.CrossEntropyLoss()

In [0]:
# Model initialization
vgg_conv_layers = [64, 64, 'pooling', 
                   128, 128, 'pooling'
                  #  , 
                  #  256, 256, 256, 'pooling', 
                  #  512, 512, 512, 'pooling', 
                  #  512, 512, 512, 'pooling'
                   ]

# Initial channel-three channels
vgg16 = VGG(make_layers(vgg_conv_layers, in_channels=3))



In [0]:
# Whether to use GPU
if use_gpu:
    vgg16 = vgg16.cuda()

penalty_ = 'CRLBO' #Penalty = 'SGD', 'CRLBO', 'L2'
crlbo = CramerRaoLowerBound(vgg16, crit=crit, penalty = penalty_, weight_decay_param = 0.1, lr=1e-4, weight=1000000)
print('Model Specs: ', crlbo.model.train())

Model Specs:  VGG(
  (conv_features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=73728, out_features=512, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Training the model

In [0]:
import warnings
warnings.filterwarnings("ignore")

model_files_path = r'\content\drive\My Drive\Colab Notebooks\Models'
# crlbo.save(model_files_path + '\\' + penalty_ + '_batch_' + str(10011) + '.h5')

In [0]:
from tqdm import tqdm

i = 0
j = 0
result = {} 

for images, labels in tqdm(train_loader):
    images = Variable(images)
    labels = Variable(labels)
    
    print('\n' , 'Train Batch -  ' , i, '--', np.bincount(labels) , 'Train Batch size: ' , images.shape[0])
    
    train_key = 'train_batch_' + str(i)
    result[i] = train_key

    if use_gpu:
        images = images.float().cuda()
        labels = labels.cuda()
    
    step = 50
    for mini_batch in range(0, images.shape[0], step):
        img, lbl  = images[mini_batch:mini_batch + step], labels[mini_batch:mini_batch + step]
        for epoch in range(EPOCH):
            torch.cuda.empty_cache()
            total = 0
            correct = 0
            crlbo.forward_backward_update(img, lbl, penalty_)

            # Calculate training accuracy
            y_pred = crlbo.model(img) 
            _, predicted = torch.max(y_pred.data, 1)
            total += lbl.size(0)
            correct += (predicted == lbl.data).sum()
            
    if epoch == EPOCH-1:
        print('Accuracy of the model on the train images: %d %%' % (100 * correct / total), 'Epoch : ', epoch, 'Mini Batch processed - ' , mini_batch , ' /', images.shape[0] )

    with torch.no_grad():
        test_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size_, sampler=BalancedBatchSampler(testset, list(testset.class_to_idx.values())), shuffle=False, num_workers = 4)
        for test_images, test_labels in tqdm(test_loader):
          
            # Test
            model = crlbo.model.eval()

            correct = 0
            total = 0

            if use_gpu:
                test_images = test_images.cuda()
                test_labels = test_labels.cuda()
            y_pred = model(test_images)
            _, predicted = torch.max(y_pred.data, 1)
            total += test_labels.size(0)
            temp = (predicted == test_labels.data).sum()
            correct += temp

            acc = (100 * correct / total)
            test_key = 'test_batch_' + str(j)
            result[j] = test_key
            result[train_key+'_'+test_key] = acc

            print('\n' , 'Test Batch ', j, ' : Accuracy: %d %%' % acc)
            
            if(i == j):
              break

            j = j + 1  
    

    i = i + 1
    j = 0

    if penalty_ == 'CRLBO':
      crlbo.register_crlbo_params(trainset, 50, 10)
    
    # crlbo.save(model_files_path + '\\' + penalty_ + '_batch_' + str(i) + '.h5')
    

0it [00:00, ?it/s]


 Train Batch -   0 -- [250 250 250 250 250 250 250 250 250 250] Train Batch size:  2500
Accuracy of the model on the train images: 8 % Epoch :  99 Mini Batch processed -  2450  / 2500



0it [00:00, ?it/s]


 Test Batch  0  : Accuracy: 9 %



1it [03:55, 235.33s/it]

...






7it [00:02,  3.74it/s]




8it [00:02,  4.17it/s]


 Test Batch  6  : Accuracy: 9 %

 Test Batch  7  : Accuracy: 8 %







9it [00:02,  4.47it/s]




10it [00:02,  4.77it/s]


 Test Batch  8  : Accuracy: 10 %

 Test Batch  9  : Accuracy: 9 %

 Test Batch  10  : Accuracy: 9 %


11it [35:04, 186.74s/it]

In [0]:

i = 0
dict_result = {}
list_result = []
for key in result:
    list_result.append(result[key])
    
    if(key == i):
        dict_result[i] = list_result
        list_result = []
        i= i + 1

dict_result[i] = list_result
            

In [0]:
print(pd.DataFrame.from_dict(dict_result, orient='index'))

                             0  ...                          10
0                 test_batch_0  ...                        None
1   tensor(9, device='cuda:0')  ...                        None
2   tensor(9, device='cuda:0')  ...                        None
3   tensor(9, device='cuda:0')  ...                        None
4   tensor(9, device='cuda:0')  ...                        None
5   tensor(9, device='cuda:0')  ...                        None
6   tensor(9, device='cuda:0')  ...                        None
7   tensor(9, device='cuda:0')  ...                        None
8   tensor(9, device='cuda:0')  ...                        None
9   tensor(9, device='cuda:0')  ...                        None
10  tensor(9, device='cuda:0')  ...               test_batch_10
11  tensor(9, device='cuda:0')  ...  tensor(9, device='cuda:0')

[12 rows x 11 columns]


In [0]:
pd.DataFrame.from_dict(dict_result, orient='index').to_csv(penalty_ + '_results.csv')